In [1]:
#YANK BP
import textwrap, yaml, yank, sys, os, glob, shutil, openmm, rdkit
#import nglview
import openmmtools as mmtools
import yank.utils
import numpy as np
from yank.experiment import YankLoader, YankDumper
import netCDF4 as nc
import matplotlib.pyplot as plt
from mdtraj.formats.dcd import DCDTrajectoryFile
import mdtraj as md
import MDAnalysis as mda
#OPENFF
from openff.units import Quantity, unit
from openmm import unit as openmm_unit
from pdbfixer import PDBFixer
from openff.toolkit import ForceField, Molecule, Topology
from openbabel import openbabel

from ybp_utils import *

In [2]:
# Split Crystal into protein and ligand pdbs based on selection strings
receptor_path, ligand_path = seperate_crys_using_MDA('3mxf.pdb', 'resname JQ1')

/home/exouser/miniconda3/envs/openff/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [3]:
# Get the filename that the next function will create for the protonated ligand in orignal format
ligand_4_solvation_path = os.path.splitext(ligand_path)[0] + '_H' + os.path.splitext(ligand_path)[-1]
# Convert ligand pdb above to an sdf with openbabel, addhs and make sdf
ligand_path = obabel_conversion((ligand_path,'ligand.sdf'))
#Verify
print(os.path.isfile(ligand_4_solvation_path), ligand_4_solvation_path, os.path.isfile(ligand_path), ligand_path)

56 Atoms 59 Bonds 1 Residues
56 Atoms 59 Bonds 1 Residues
True ligand_H.pdb True ligand.sdf


In [4]:
# Protonate the receptor (ph7)
receptor_path = protonate_with_pdb2pqr(receptor_path, 'receptor_H.pdb')

Protanting using command line
pdb2pqr30 --ff AMBER --nodebump --keep-chain --ffout AMBER --pdb-output receptor_H.pdb --with-ph 7 receptor.pdb receptor_H.pqr


INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: receptor.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 127 residues and 1059 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 3 missing atoms in biomolecule.
INFO:Added atom CD to residue LYS A 141 at coordinates 46.066, 36

In [5]:
# Load a molecule from a SDF file
ligand = Molecule.from_file(ligand_path)

# Print out a SMILES code for the ligand
print(ligand.to_smiles(explicit_hydrogens=False))

Cc1sc2c(c1C)C(c1ccc(Cl)cc1)=N[C@@H](CC(=O)OC(C)(C)C)c1nnc(C)n1-2


In [6]:
ligand_solvated_path = PDBFixer_solvation(ligand_4_solvation_path)

/home/exouser/miniconda3/envs/openff/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM   33  H   JQ1 A   1      36.571  37.893  36.614  1.00  0.00           H  , HETATM   32  H   JQ1 A   1      37.726  39.146  36.226  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/exouser/miniconda3/envs/openff/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (HETATM   34  H   JQ1 A   1      36.202  39.068  35.374  1.00  0.00           H  , HETATM   33  H   JQ1 A   1      36.571  37.893  36.614  1.00  0.00           H  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/exouser/miniconda3/envs/openff/lib/python3.9/site-packages/openmm/app/internal/pdbstructure.py

HELLO


In [7]:
receptor_solvated_fn = PDBFixer_solvation(receptor_path)

HELLO


In [23]:
# Openff function to add the ligand and remove clashing waters
comp_top = Topology.from_pdb(receptor_solvated_fn)
comp_top = insert_molecule_and_remove_clashes(comp_top, ligand)

# Save the TOP opbject to a json file to load easier in the future
with open("complex_topology.json", "w") as f:
    print(comp_top.to_json(), file=f)

Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule


In [24]:
print(ligand_solvated_path)
u = mda.Universe(ligand_solvated_path)
waters = u.select_atoms('resname HOH')

ligand_H_solvated.pdb


In [25]:
waters.write('solvent.pdb')

/home/exouser/miniconda3/envs/openff/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [26]:
#Solv top must be built from a file with only waters, and then the ligand can be added?
solv_top = Topology.from_pdb('solvent.pdb')
solv_top = insert_molecule_and_remove_clashes(solv_top, ligand)

Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule
Removed [H][O][H] molecule


In [27]:
sage_ff14sb = ForceField("openff-2.1.0.offxml", "opc-1.0.1.offxml", "ff14sb_off_impropers_0.0.3.offxml")

In [28]:
#THIS WILL (potentially) BE VERY USEFUL CLASS, see https://docs.openforcefield.org/projects/interchange/en/stable/using/output.html
comp_interchange = sage_ff14sb.create_interchange(comp_top)
solv_interchange = sage_ff14sb.create_interchange(solv_top)

In [29]:
#Write a pdb of the system
comp_interchange.to_pdb("system.pdb")
solv_interchange.to_pdb("solvent.pdb")

In [30]:
openmm_comp = make_openmm_simulation(comp_interchange)
openmm_solv = make_openmm_simulation(solv_interchange)

Original state has energy nan kJ/mol  with maximum force 3719544752.54 kJ/(mol nm)
Minimized state has energy -517538.47 kJ/mol  with maximum force 4021.83 kJ/(mol nm)
Original state has energy nan kJ/mol  with maximum force 3719549457.77 kJ/(mol nm)
Minimized state has energy -24137.67 kJ/mol  with maximum force 3622.43 kJ/(mol nm)


In [33]:
with open('complex_final.xml', "w") as xml_file:
    xml_file.write(openmm.XmlSerializer.serialize(openmm_comp.system))

with open('complex_final.pdb', "w") as pdb_file:  
    openmm.app.PDBFile.writeFile(openmm_comp.topology, openmm_comp.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions(),
      file=pdb_file, keepIds=True)

In [34]:
with open('solvent_final.xml', "w") as xml_file:
    xml_file.write(openmm.XmlSerializer.serialize(openmm_solv.system))

with open('solvent_final.pdb', "w") as pdb_file:  
    openmm.app.PDBFile.writeFile(openmm_solv.topology, openmm_solv.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions(),
      file=pdb_file, keepIds=True)

In [ ]:
##Write AMBER Files using INTERCHANGE
#interchange.to_prmtop("system.prmtop")
#interchange.to_inpcrd("system.inpcrd")

In [ ]:
##Write GROMACS file using INTERCHANGE
#interchange.to_gro("system.gro")
#interchange.to_top("system.top")

In [ ]:
run_for_steps(make_openmm_simulation(interchange), 10000)

In [ ]:
#MOST SIMULATIONS STARTED FROM THE AMBER FILES ARE GOING NAN

# from openmm.app import *
# from openmm import *
# from openmm.unit import *
# from sys import stdout

# prmtop = AmberPrmtopFile('system.prmtop')
# inpcrd = AmberInpcrdFile('system.inpcrd')

In [ ]:
# system = prmtop.createSystem()
# integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.001*picoseconds)
# simulation = Simulation(prmtop.topology, system, integrator)
# simulation.context.setPositions(inpcrd.positions)
# if inpcrd.boxVectors is not None:
#     simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
# simulation.minimizeEnergy()
# simulation.reporters.append(PDBReporter('output.pdb', 1000))
# simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
#         potentialEnergy=True, temperature=True))
# simulation.step(10000)

In [ ]:
# #u = mda.Universe('system.prmtop', topology_format='PARM7')
# import parmed as pmd

# fmt = pmd.amber.AmberFormat('system.prmtop')
# print(fmt.parm_data['POINTERS'][0]) # number of atoms
# print(len(fmt.parm_data['ATOM_NAME']))
# print(len(fmt.parm_data['RADII']))